#USE AI AGENTs, like AUTOGPT or BABYAGI, without paids API. TOTALLY FOR FREE 🎁

![AUTO GPT FREE](https://static.wixstatic.com/media/3c029f_7db5f44379074783b6f2de67b82eb3b5~mv2.gif)

## SETUP REPOSITORY 📂


In [ ]:
!git clone https://github.com/IntelligenzaArtificiale/Free-AUTO-GPT-with-NO-API.git


In [ ]:
import pathlib
import re
import subprocess
import typing


def is_in_jupyter_notebook() -> bool:
    """It checks whether a Jupyter notebook is being run"""
    try:
        get_ipython
        return True
    except NameError:
        return False


def is_on_gcolab() -> bool:
    """It checks whether a Jupyter notebook is being run on Google Colab"""
    if not is_in_jupyter_notebook():
        return False

    return "google.colab" in str(get_ipython())


def is_ubuntu_20_04() -> bool:
    import lsb_release
    metadata = lsb_release.get_os_release()

    distro  = metadata["ID"].lower()
    release = metadata["RELEASE"]

    return distro == "ubuntu" and release == "20.04"


def setup_ubuntu_20_04() -> None:
    """It sets up a Ubuntu 20.04 container with `chromium-browser`

    For more information, see 
    https://github.com/googlecolab/colabtools/issues/3347#issuecomment-1387453484
    """
    # It adds debian buster
    EOF_debian_buster = """\
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
"""
    !echo "$EOF_debian_buster" > /etc/apt/sources.list.d/debian.list

    # It adds keys
    !apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
    !apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
    !apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

    !apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
    !apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
    !apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

    # It adds the debian repo for chromium* packages only
    # Note the double-blank lines between entries
    EOF_chromium_pref = """\
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
"""
    !echo "$EOF_chromium_pref" > /etc/apt/preferences.d/chromium.pref

    # It installs the packages
    !apt-get update
    !apt-get install chromium chromium-driver
    !apt-get install -y xvfb


def setup_requirements() -> None:
    PIP_requirements = " ".join([
        "PyVirtualDisplay", # To run a virtual display
        "undetected-chromedriver==3.2.1",
    ])

    !python3 -m pip install --upgrade $PIP_requirements


def get_py_module_path(module: str) -> typing.Optional[pathlib.Path]:
    """It gets the absolute path of a Python module"""
    r = subprocess.run(
        ["pip", "show", module], 
        capture_output=True
    )

    try:
        r.check_returncode()
    except subprocess.CalledProcessError:
        return None

    stdout = r.stdout.decode()

    try:
        RE_abspath = "\nLocation: (?P<abspath>.*)\n"

        matches = re.search(RE_abspath, stdout)
        abspath = matches.group("abspath")
    except AttributeError:
        return None

    dist_packages = pathlib.Path(abspath).resolve()
    return dist_packages / module


def patch_undetected_chromedriver() -> None:
    """It forces `undetected_chromedriver` to run the Chromium webdriver

    For more information, see 
    https://github.com/ultrafunkamsterdam/undetected-chromedriver/issues/108#issuecomment-1170269377
    """
    chromedriver_filename = "chromedriver_linux64.zip"

    src_chromedriver_filepath = ROOT / chromedriver_filename
    dst_chromedriver_filepath = pathlib.Path("/tmp") / chromedriver_filename

    !zip -j "$src_chromedriver_filepath" /usr/bin/chromedriver

    PY_module = "undetected_chromedriver"
    module_path = get_py_module_path(PY_module)

    patcher_filepath = module_path / "patcher.py"

    with patcher_filepath.open("rt") as f:
        contents = f.read()

    src = f"'file://{src_chromedriver_filepath}'"
    dst = f"'{dst_chromedriver_filepath}'"

    # It is forced to use the local webdriver
    contents = contents.replace(
        f"return urlretrieve(u)[0]",
        f"return urlretrieve({src}, filename={dst})[0]"
    )

    with patcher_filepath.open("wt") as f:
        f.write(contents)


def setup_container() -> None:
    """It sets up the container which is being run"""
    if is_ubuntu_20_04():
        setup_ubuntu_20_04()

    setup_requirements()
    patch_undetected_chromedriver()


ROOT = pathlib.Path("/content")
anchor = ROOT / "anchor.txt"


assert is_on_gcolab(), "It seems you are not on Google Colab"

# It will set the Google Colab container up only
# after disconnections, not after restarts
if not anchor.exists():
    setup_container()
    anchor.touch()

In [ ]:
%pip install -r requirements.txt 
# install dependencies
%pip install PyVirtualDisplay


## HOW TO GET TOKEN and Cookies FREE 💡
 



- **HUGGINGFACE🤗** : Visit this simple [official guide](https://huggingface.co/docs/hub/security-tokens)

- **Now Running also with HuggingCHAT**

- **(OPTIONAL BUT BETTER RESULT) CHATGPT🖥** : 
    1. Go to https://chat.openai.com/chat and open the developer tools by `F12`.
    2. Find the `__Secure-next-auth.session-token` cookie in `Application` > `Storage` > `Cookies` > `https://chat.openai.com`.
    3. Copy the value in the `Cookie Value` field.


![image](https://user-images.githubusercontent.com/19218518/206170122-61fbe94f-4b0c-4782-a344-e26ac0d4e2a7.png)

- **(OPTIONAL) Google Bard🖥** : 
    1. Go toGo to https://bard.google.com/ and open the developer tools by `F12`.
    2. Find the `__Secure-1PSID` cookie in `Application` > `Storage` > `Cookies` 
    3. Copy the value in the `Cookie Value` field.

![Cattura](https://user-images.githubusercontent.com/108482353/236518416-ba0fb89c-080d-4e5e-8514-4ed7ac897b55.PNG)



- **(OPTIONAL) Bing CHAT** :
    1. Check if you have access to [Bing Chat](https://chrome.google.com/webstore/detail/bing-chat-for-all-browser/jofbglonpbndadajbafmmaklbfbkggpo)
    2. Install the cookie editor extension for [Chrome](https://chrome.google.com/webstore/detail/cookie-editor/hlkenndednhfkekhgcdicdfddnkalmdm) or [Firefox](https://addons.mozilla.org/en-US/firefox/addon/cookie-editor/)
    3. Go to `bing.com` 
    4. Open the extension for cookie
    5. Click "Export" on the bottom right, then "Export as JSON" (This saves your cookies to clipboard)
    6. Paste your cookies into a file `/content/Free-AUTO-GPT-with-NO-API/cookiesBing.json`


![image](https://user-images.githubusercontent.com/108482353/236259872-faf7946c-5648-4733-8d66-978040eacd85.png)


## 🚀SAVE HERE YOURS TOKENs🚀



In [ ]:
import os

# Hugging FACE
os.environ["HUGGINGFACE_TOKEN"] = "COPY-HERE-TOKEN"

# ChatGPT ( NOT WORKING ON GOOGLE COLAB )
os.environ["CHATGPT_TOKEN"] = "COPY-HERE-TOKEN"

# BING chat ( rember to copy yours cookies into : /content/Free-AUTO-GPT-with-NO-API/cookiesBing.json )
os.environ["BINGCHAT_COOKIEPATH"] = "/content/Free-AUTO-GPT-with-NO-API/cookiesBing.json"

# Google Bard
os.environ["BARDCHAT_TOKEN"] = "COPY-HERE-COOKIE"



# ⚠️ DONT RUN **CHATGPT** ISTANCE ON COLAB DOSENT WORK FOR NOW ⚠️ 
# ⚠️ USE ONLY HUGGING CHAT ⚠️ 

## RUN Other Agent or Ddevelop your own Agent 👨‍💻 🧑‍💻

In [ ]:
# Check for available AGENT
!ls /content/Free-AUTO-GPT-with-NO-API/OtherAgent/



In [ ]:
# open /content/Free-AUTO-GPT-with-NO-API/OtherAgent/FreeLLM/pyChatGPT.py
# 


!python /content/Free-AUTO-GPT-with-NO-API/OtherAgent/csvAgent.py

#insert this path for csv : /content/Free-AUTO-GPT-with-NO-API/OtherAgent/startup.csv

In [ ]:
!python /content/Free-AUTO-GPT-with-NO-API/OtherAgent/pythonAgent.py

In [ ]:
!python /content/Free-AUTO-GPT-with-NO-API/OtherAgent/customAgent.py 

## RUN BABYAGI 👶

In [ ]:
#Open the /content/Free-AUTO-GPT-with-NO-API/BABYAGI.py and Set "verbose=True" for view the debug 

!python /content/Free-AUTO-GPT-with-NO-API/BABYAGI.py 

## RUN Meta-Prompt 👥

In [ ]:
!python /content/Free-AUTO-GPT-with-NO-API/MetaPrompt.py

## RUN AUTOGPT 🤖

In [ ]:
#Open the /content/Free-AUTO-GPT-with-NO-API/BABYAGI.py and modify or add new tool 
"""
#ACTUAL TOOL 
tools = [
    web_search, #DuckDuckGoSearchRun()
    WriteFileTool(root_dir="/data"),
    ReadFileTool(root_dir="/data"),
    process_csv,
    query_website_tool, playwright()
    # HumanInputRun(), # Activate if you want the permit asking for help from the human
]
"""
!python /content/Free-AUTO-GPT-with-NO-API/AUTOGPT.py

## RUN CAMEL with Streamlit.io 🐪

---


In [ ]:
# SETUP COLAB for run Streamlit
!npm install localtunnel

In [ ]:
!streamlit run /content/Free-AUTO-GPT-with-NO-API/Camel.py &>/content/logs.txt &
!npx localtunnel --port 8501